In [ ]:
import sqlite3
import time
import random
import numpy as np

def solve_mTSP_ant_colony(instance_id, db_file, num_ants=20, alpha=1.0, beta=2.0, evaporation_rate=0.5, stagnation_limit=20, time_limit=100):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Instance details
    cursor.execute("SELECT nr_cities, nr_salesmen FROM instances WHERE instance_id = ?", (instance_id,))
    instance = cursor.fetchone()
    if not instance:
        print(f"Instance {instance_id} not found.")
        conn.close()
        return

    nr_cities, nr_salesmen = instance

    # City coordinates
    cursor.execute("SELECT city_id, x, y FROM cities WHERE instance_id = ?", (instance_id,))
    cities = cursor.fetchall()

    # Create the distance matrix
    distance_matrix = np.array([[
        ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5 for _, x2, y2 in cities
    ] for _, x1, y1 in cities])

    # Initialize pheromone matrix
    pheromone_matrix = np.ones((nr_cities + 1, nr_cities + 1))

    # Ant Colony Optimization
    best_routes = None
    best_cost = float('inf')
    stagnation_counter = 0  # Counter for stagnation

    start_time = time.time()

    iteration = 0
    while True:
        all_routes = []
        all_costs = []

        for ant in range(num_ants):
            routes = [[] for _ in range(nr_salesmen)]
            visited = set()
            visited.add(0)  # Depot is always visited

            # Assign at least one city to each salesman
            unvisited_cities = set(range(1, nr_cities + 1))
            for salesman_id in range(nr_salesmen):
                if unvisited_cities:
                    probabilities = [
                        (pheromone_matrix[0][city] ** alpha) * ((1 / (distance_matrix[0][city] + 1e-6)) ** beta)
                        for city in unvisited_cities
                    ]
                    probabilities = np.array(probabilities) / sum(probabilities)
                    nearest_city = random.choices(list(unvisited_cities), weights=probabilities, k=1)[0]
                    routes[salesman_id].append(0)  # Start at depot
                    routes[salesman_id].append(nearest_city)
                    visited.add(nearest_city)
                    unvisited_cities.remove(nearest_city)

            # Assign remaining cities
            while unvisited_cities:
                for salesman_id, route in enumerate(routes):
                    if not unvisited_cities:
                        break
                    current_city = route[-1]
                    probabilities = [
                        (pheromone_matrix[current_city][city] ** alpha) * ((1 / (distance_matrix[current_city][city] + 1e-6)) ** beta)
                        for city in unvisited_cities
                    ]
                    probabilities = np.array(probabilities) / sum(probabilities)
                    next_city = random.choices(list(unvisited_cities), weights=probabilities, k=1)[0]
                    route.append(next_city)
                    visited.add(next_city)
                    unvisited_cities.remove(next_city)

            # All salesmen return to the depot
            for route in routes:
                if route and route[-1] != 0:
                    route.append(0)

            # Calculate cost for this ant
            total_cost = sum(
                sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
                for route in routes
            )
            all_routes.append(routes)
            all_costs.append(total_cost)

            # Update best solution
            if total_cost < best_cost:
                best_cost = total_cost
                best_routes = routes
                stagnation_counter = 0  # Reset stagnation counter
            else:
                stagnation_counter += 1

        # Update pheromone matrix
        pheromone_matrix *= (1 - evaporation_rate)
        for routes, cost in zip(all_routes, all_costs):
            for route in routes:
                for i in range(len(route) - 1):
                    pheromone_matrix[route[i]][route[i + 1]] += 1 / cost

        # Check for stagnation
        if stagnation_counter >= stagnation_limit:
            pheromone_matrix += np.random.rand(nr_cities + 1, nr_cities + 1) * 0.1
            stagnation_counter = 0

        # Check time limit
        current_time = time.time()
        if current_time - start_time >= time_limit:
            print(f"Time limit of {time_limit} seconds reached. Returning best solution found.")
            break

        iteration += 1

    # Final metrics
    end_time = time.time()
    time_taken = end_time - start_time

    # Calculate distance gap
    distances = [
        sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        for route in best_routes
    ]
    distance_gap = max(distances) - min(distances)

    # Calculate efficiency
    epsilon = 1e-6  # offset for division by zero
    efficiency = best_cost / (time_taken + epsilon)

    # Insert results into the algorithms table
    cursor.execute("""
        INSERT OR REPLACE INTO algorithms (instance_id, strategy, total_cost, time_taken, distance_gap, efficiency)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (instance_id, "Ant Colony", best_cost, time_taken, distance_gap, efficiency))

    # Clear previous routes and insert routes into the routes table
    cursor.execute("DELETE FROM routes WHERE instance_id = ? AND strategy = ?", (instance_id, "Ant Colony"))

    for salesman_id, route in enumerate(best_routes):
        cursor.execute("""
            INSERT INTO routes (instance_id, strategy, salesman_id, route)
            VALUES (?, ?, ?, ?)
        """, (instance_id, "Ant Colony", salesman_id, str(route)))

    conn.commit()
    conn.close()

    print(f"Instance {instance_id} solved using Ant Colony Optimization.")


# Solve for each instance
def solve_mTSP_ant_colony_for_all_instances(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Select instances that have not been solved with Ant Colony
    cursor.execute("""
        SELECT instance_id 
        FROM instances 
        WHERE instance_id NOT IN (
            SELECT instance_id 
            FROM algorithms 
            WHERE strategy = 'Ant Colony'
        )
    """)
    instances = cursor.fetchall()

    if not instances:
        print("No unsolved instances found for Ant Colony.")
        conn.close()
        return

    for (instance_id,) in instances:
        solve_mTSP_ant_colony(instance_id, db_file)

    conn.close()

    


# Example usage
solve_mTSP_ant_colony_for_all_instances(db_file="../train_mTSP.sqlite3")

KeyboardInterrupt: 